In [340]:
import json as js
from tqdm import tqdm
import string
import sys
import keras
import tensorflow as tf
import json
import numpy as np
import os
import dask.array as da
os.environ["CUDA_DEVICE_ORDER"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras.layers import Input,Dropout,Dense,Embedding,LSTM,add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer as Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [341]:
if "all_descriptions_train.zarr" in os.listdir('.') and "embedding_train_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_validation = da.from_zarr("embedding_train_128.zarr")
        desc_validation = da.from_zarr("all_descriptions_train.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

if "all_descriptions_validation.zarr" in os.listdir('.') and "embedding_validation_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_validation = da.from_zarr("embedding_validation_128.zarr")
        desc_validation = da.from_zarr("all_descriptions_validation.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

if "all_descriptions_test.zarr" in os.listdir('.') and "embedding_test_128.zarr" in os.listdir('.'):
    with tf.device('/cpu:0'):
        embedding_test = da.from_zarr("embedding_test_128.zarr")
        desc_test = da.from_zarr("all_descriptions_test.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

In [342]:
embedding_train = embedding_train[:100]
embedding_validation = embedding_validation[:20]
embedding_test = embedding_test[:10]
desc_test

dask.array<from-zarr, shape=(1000,), dtype=<U121, chunksize=(827,), chunktype=numpy.ndarray>

In [343]:
allText = np.append(desc_train.compute(),desc_validation.compute())
allText = np.append(allText,desc_test.compute())
tokenizer = Tokenizer()
tokenizer.fit_on_texts(allText)
vocab_size = len(tokenizer.word_index) + 1

In [344]:
max_length = max(len(d.split()) for d in allText)
max_length

32

In [345]:
def count_length(tokenizer, descriptions):
    Y = 0
    for i in range(len(descriptions)):
        seq = tokenizer.texts_to_sequences([(descriptions[i].compute()).tolist()])[0]
        Y+=len(seq)-1
    return Y

In [346]:
nb_train_samples = count_length(tokenizer, desc_train)
nb_validation_samples = count_length(tokenizer, desc_validation)
print(nb_train_samples, nb_validation_samples)

48255 19277


In [347]:
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(80,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [348]:
model = define_model(vocab_size, max_length)

epochs = 50

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 32, 256)      1081088     input_16[0][0]                   
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 80)           0           input_15[0][0]                   
____________________________________________________________________________________________

In [349]:
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
    while 1:
        for i in range(len(descriptions)):
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, descriptions[i], photos[i], vocab_size)
            nb = in_img.shape[0]
            yield [[in_img, in_seq], out_word, nb]

In [352]:
with tf.device('/gpu:0'):
    stop_early = EarlyStopping(monitor='val_loss',patience = 10)
#     reduceLR = ReduceLROnPlateau(monitor='val_loss',paitence = 20, factor=0.2, min_lr = 0.0001)
    ModelCheck = ModelCheckpoint('best_model_rnn.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    callbacks = [stop_early, ModelCheck]
    for i in range(epochs):
        train_generator = data_generator(desc_train, embedding_train, tokenizer, max_length, vocab_size)
        validation_generator = data_generator(desc_validation, embedding_validation, tokenizer, max_length, vocab_size)
        i_train, o_train, batch_size_train = next(train_generator)
        i_validation, o_validation, batch_size_validation = next(validation_generator)
        model.fit(
            x = i_train, 
            y = o_train, 
            epochs=1, 
            steps_per_epoch=nb_train_samples//batch_size_train, 
            verbose=1, 
            validation_data=(i_validation, o_validation), 
            validation_steps=nb_validation_samples//batch_size_validation,
            callbacks=callbacks)

5360/5361 [============================>.] - ETA: 0s - loss: 0.0480
Epoch 00001: val_loss improved from inf to 47.39310, saving model to best_model_rnn.h5
5360/5361 [============================>.] - ETA: 0s - loss: 2.1810e-07
Epoch 00001: val_loss improved from 47.39310 to 46.87392, saving model to best_model_rnn.h5
 104/5361 [..............................] - ETA: 9:30 - loss: 2.9802e-08

KeyboardInterrupt: 

In [368]:
model = load_model('best_model_rnn.h5')

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(rnn_model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
#     print(type(photo))
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        print(sequence)
#         print(np.array([photo, sequence]).shape)
        yhat = rnn_model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

generate_desc(model, tokenizer, embedding_test[0].compute(), max_length)
# embedding_test[0].compute().shape

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


ValueError: Error when checking input: expected input_15 to have shape (80,) but got array with shape (1,)